<a href="https://colab.research.google.com/github/MatCif/Tesis/blob/main/Tesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lectura de datos

In [1]:
import pandas as pd

# Use the 'on_bad_lines' argument to handle lines that cause errors in the first file
bdh_historica_df = pd.read_csv('/content/BDH_HISTORICA.csv', on_bad_lines='skip', sep=';') # Use 'skip' to skip bad lines

# Use the 'on_bad_lines' argument to handle lines that cause errors in the second file
postulaciones_df = pd.read_csv('/content/POSTULACIONES_2016_2024.csv', on_bad_lines='skip', sep=';')


# Cargar el archivo de Excel
dfs = pd.read_excel('/content/03_BaseINDICES_Indicadores (1).xlsx', sheet_name=None)

# Crear una lista para almacenar los DataFrames
all_data = []

# Iterar sobre las hojas del Excel
for sheet_name, df in dfs.items():
  # Derretir el DataFrame para que cada indicador esté en una fila separada
  df_melted = pd.melt(df, id_vars=['Institución', 'Año'], var_name='Indicador', value_name='Valor')

  # Agregar columna para la hoja (renombrada a Dimensión)
  df_melted['Dimensión'] = sheet_name

  # Agregar el DataFrame a la lista
  all_data.append(df_melted)

# Concatenar todos los DataFrames en uno solo
consolidated_df = pd.concat(all_data, ignore_index=True)

# Reordenar las columnas
consolidated_df = consolidated_df[['Institución', 'Año', 'Dimensión', 'Indicador', 'Valor']]

# Eliminar filas con valores faltantes en "Institución"
consolidated_df = consolidated_df.dropna(subset=['Institución'])

# Mostrar el DataFrame consolidado
#print(consolidated_df)


print("CONSOLIDADO UES")
consolidated_df.columns
print("bdh_historica")
bdh_historica_df.columns
print("postulaciones_df")
postulaciones_df.columns



CONSOLIDADO UES
bdh_historica
postulaciones_df


Index(['N', 'CODIGO_PROYECTO', 'SUBDIRECCION', 'PROGRAMA', 'INSTRUMENTO',
       'NOMBRE_CONCURSO', 'ESTADO_RESOLUCION_CONCURSO', 'DISCIPLINA_OECD',
       'REGION_EJECUCION', 'TIPO_BENEFICIARIO', 'TIPO_INSTITUCION', 'SEXO',
       'AGNO_FALLO', 'MACROZONA_MINCIENCIA'],
      dtype='object')

In [2]:
for sheet_name, df in dfs.items():
  print(f"DataFrame: {sheet_name}")
  print(df.info())
  print("\n")

print("DataFrame: bdh_historica_df")
print(bdh_historica_df.info())
print("\n")

print("DataFrame: postulaciones_df")
print(postulaciones_df.info())
print("\n")

DataFrame: 1_ContextoInstitucional
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518 entries, 0 to 2517
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Año                               2518 non-null   int64  
 1   Clasificación                     2518 non-null   object 
 2   Institución                       2518 non-null   object 
 3   Matrícula Total                   2518 non-null   int64  
 4   Matrícula Total 1er año           2518 non-null   int64  
 5   Vacantes 1er semestre             1963 non-null   float64
 6   N° Áreas                          2518 non-null   int64  
 7   N° Carrera Genérica               2518 non-null   int64  
 8   N° Programas                      2518 non-null   int64  
 9   N° Programas Ofrecidos            1963 non-null   float64
 10  N° Titulados                      2237 non-null   float64
 11  N° Académicos                     

Una vez hecho extraído los datos se  deben normalizar los datos  para luego consolidarlos y trabajar en ellos

In [3]:
import re

def normalize_name(name):
    if pd.isna(name):
        return name
    name = name.lower().strip()
    name = re.sub(r'\s+', ' ', name)  # Reemplaza múltiples espacios por uno
    name = name.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    name = name.replace('ü', 'u').replace('ñ', 'n')
    # Eliminar caracteres especiales
    name = ''.join(e for e in name if e.isalnum() or e.isspace())
    return name



In [4]:
institution_mapping = {
    # Universidad de Chile
    'u de chile': 'uchile',
    'universidad de chile': 'uchile',

    # Pontificia Universidad Católica de Chile
    'pontificia u catolica de chile': 'uc',
    'pontificia universidad catolica de chile': 'uc',

    # Pontificia Universidad Católica de Valparaíso
    'pontificia u catolica de valparaiso': 'ucv',
    'universidad catolica de valparaiso': 'ucv',

    # Universidad de Santiago de Chile
    'u de santiago de chile': 'usach',
    'universidad de santiago de chile': 'usach',

    # Universidad Técnica Federico Santa María
    'u tecnica federico santa maria': 'utfsm',
    'universidad tecnica federico santa maria': 'utfsm',

    # Universidad de Concepción
    'u de concepcion': 'udec',
    'universidad de concepcion': 'udec',

    # Universidad Austral de Chile
    'u austral de chile': 'uach',
    'universidad austral de chile': 'uach',

    # Universidad de Valparaíso
    'u de valparaiso': 'uv',
    'universidad de valparaiso': 'uv',

    # Universidad Católica del Norte
    'u catolica del norte': 'ucn',
    'universidad catolica del norte': 'ucn',

    # Universidad de La Frontera
    'u de la frontera': 'ufro',
    'universidad de la frontera': 'ufro',

    # Universidad de Antofagasta
    'u de antofagasta': 'uantof',
    'universidad de antofagasta': 'uantof',

    # Universidad de Tarapacá
    'u de tarapaca': 'uta',
    'universidad de tarapaca': 'uta',

    # Universidad de Magallanes
    'u de magallanes': 'umag',
    'universidad de magallanes': 'umag',

    # Universidad de Los Lagos
    'u de los lagos': 'ulagos',
    'universidad de los lagos': 'ulagos',

    # Universidad de O'Higgins
    'u de o higgins': 'uoh',
    'universidad de ohiggins': 'uoh',

    # Universidad de Playa Ancha
    'u de playa ancha de ciencias de la educacion': 'upa',
    'universidad de playa ancha': 'upa',

    # Universidad de Talca
    'u de talca': 'utalca',
    'universidad de talca': 'utalca',

    # Universidad del Bío-Bío
    'u del biobio': 'ubiobio',
    'universidad del biobio': 'ubiobio',

    # Universidad Metropolitana de Ciencias de la Educación
    'u metropolitana de ciencias de la educacion': 'umce',
    'universidad metropolitana de ciencias de la educacion': 'umce',

    # Universidad de Aysén
    'u de aysen': 'uaysen',
    'universidad de aysen': 'uaysen',

    # Universidad de Los Andes
    'u de los andes': 'uandes',
    'universidad de los andes': 'uandes',

    # Universidad Diego Portales
    'u diego portales': 'udp',
    'universidad diego portales': 'udp',

    # Universidad Técnica Metropolitana
    'u tecnologica metropolitana': 'utem',
    'universidad tecnologica metropolitana': 'utem',

    # Universidad Central
    'u central de chile': 'ucentral',
    'universidad central de chile': 'ucentral',

    # Universidad Santo Tomás
    'u santo tomas': 'ust',
    'universidad santo tomas': 'ust',

    # Universidad San Sebastián
    'u san sebastian': 'uss',
    'universidad san sebastian': 'uss',

    # Universidad Adolfo Ibáñez
    'u adolfo ibanez': 'uai',
    'universidad adolfo ibanez': 'uai',

    # Universidad Finis Terrae
    'u finis terrae': 'uft',
    'universidad finis terrae': 'uft',

    # Universidad Mayor
    'u mayor': 'umayor',
    'universidad mayor': 'umayor',

    # Universidad Católica de Temuco
    'u catolica de temuco': 'uct',
    'universidad catolica de temuco': 'uct',

    # Universidad Andrés Bello
    'u andres bello': 'unab',
    'universidad andres bello': 'unab',

    # Universidad Alberto Hurtado
    'u alberto hurtado': 'uah',
    'universidad alberto hurtado': 'uah',

    # Universidad del Desarrollo
    'u del desarrollo': 'udd',
    'universidad del desarrollo': 'udd',

    # Universidad de las Américas
    'u de las americas': 'udla',
    'universidad de las americas': 'udla',

    # Universidad Católica del Maule
    'u catolica del maule': 'ucm',
    'universidad catolica del maule': 'ucm',

    # Universidad Bernardo O'Higgins
    'u bernardo ohiggins': 'ubo',
    'universidad bernardo ohiggins': 'ubo',

    # Universidad Arturo Prat
    'u arturo prat': 'unap',
    'universidad arturo prat': 'unap',

    # Universidad Academia de Humanismo Cristiano
    'u academia de humanismo cristiano': 'uahc',
    'universidad academia de humanismo cristiano': 'uahc',

    # Universidad de Artes, Ciencias y Comunicación UNIACC
    'u de artes ciencias y comunicacion uniacc': 'uniacc',
    'universidad de artes, ciencias y comunicacion uniacc': 'uniacc',

    # Universidad Pedro de Valdivia
    'u pedro de valdivia': 'upv',
    'universidad pedro de valdivia': 'upv',

    # Universidad SEK
    'u sek': 'usek',
    'universidad sek': 'usek',

    # Universidad Los Leones
    'u los leones': 'uleones',
    'universidad de los leones': 'uleones',

    # Universidad de las Ciencias de la Información
    'u de rancagua': 'u_rancagua',

    # Centros de Formación Técnica (CFT) e Institutos Profesionales (IP)
    'ip aiep': 'aiep',
    'ip inacap': 'inacap',
    'cft inacap': 'inacap',
    'cft duoc uc': 'duoc_uc',
    'ip duoc uc': 'duoc_uc',
    'ip esucomex': 'esucomex',
    'cft protec': 'protec',
    'u del pacifico': 'udp',
    'cft estatal de la araucania': 'cft_araucania',

    # Otros
    'cft esucomex': 'esucomex',
    'cft san agustin de talca': 'cft_san_agustin',
    'ip santo tomas': 'ip_santo_tomas',
    'cft santo tomas': 'cft_santo_tomas',

    'u catolica cardenal raul silva henriquez': 'ucsh',
    'universidad catolica cardenal raul silva henriquez': 'ucsh',
    'u catolica de la santisima concepcion': 'uscc',
    'universidad catolica de la santisima concepcion': 'uscc'
}




In [5]:
# Iterar sobre las hojas del Excel
for sheet_name, df in dfs.items():
    # Aplicar la normalización a la columna de 'Institución'
    df['Institución'] = df['Institución'].apply(normalize_name)

    # Aplicar el mapeo de nombres de instituciones
    df['Institución'] = df['Institución'].replace(institution_mapping)

    # Derretir el DataFrame para que cada indicador esté en una fila separada
    df_melted = pd.melt(df, id_vars=['Institución', 'Año'], var_name='Indicador', value_name='Valor')

    # Agregar columna para la hoja (renombrada a Dimensión)
    df_melted['Dimensión'] = sheet_name

    # Agregar el DataFrame a la lista
    all_data.append(df_melted)

# Concatenar todos los DataFrames en uno solo
consolidated_df = pd.concat(all_data, ignore_index=True)

# Reordenar las columnas
consolidated_df = consolidated_df[['Institución', 'Año', 'Dimensión', 'Indicador', 'Valor']]

# Eliminar filas con valores faltantes en "Institución"
consolidated_df = consolidated_df.dropna(subset=['Institución'])

# Mostrar las columnas consolidadas
print("CONSOLIDADO UES")
print(consolidated_df.columns)

print("bdh_historica")
print(bdh_historica_df.columns)

print("postulaciones_df")
print(postulaciones_df.columns)

# Normalizar y mapear instituciones en los otros DataFrames
bdh_historica_df['Institución'] = bdh_historica_df['INSTITUCION_PRINCIPAL'].apply(normalize_name)
bdh_historica_df['Institución'] = bdh_historica_df['Institución'].replace(institution_mapping)

postulaciones_df['Institución'] = postulaciones_df['TIPO_INSTITUCION'].apply(normalize_name)
postulaciones_df['Institución'] = postulaciones_df['Institución'].replace(institution_mapping)


CONSOLIDADO UES
Index(['Institución', 'Año', 'Dimensión', 'Indicador', 'Valor'], dtype='object')
bdh_historica
Index(['N', 'CODIGO_PROYECTO', 'SUBDIRECCION', 'PROGRAMA', 'INSTRUMENTO',
       'NOMBRE_CONCURSO', 'AGNO_CONCURSO', 'AGNO_FALLO', 'NOMBRE_PROYECTO',
       'AREA_OCDE', 'DISCIPLINA_DETALLE', 'GRUPO_DE_EVALUACION',
       'DURACION_MESES', 'TIPO_BENEFICIARIO', 'NOMBRE_RESPONSABLE', 'SEXO',
       'INSTITUCION_PRINCIPAL', 'MACROZONA_MINCIENCIA', 'REGION_EJECUCION',
       'MONTO_ADJUDICADO', 'NOTA_MONTO', 'MONEDA', 'PALABRAS_CLAVES'],
      dtype='object')
postulaciones_df
Index(['N', 'CODIGO_PROYECTO', 'SUBDIRECCION', 'PROGRAMA', 'INSTRUMENTO',
       'NOMBRE_CONCURSO', 'ESTADO_RESOLUCION_CONCURSO', 'DISCIPLINA_OECD',
       'REGION_EJECUCION', 'TIPO_BENEFICIARIO', 'TIPO_INSTITUCION', 'SEXO',
       'AGNO_FALLO', 'MACROZONA_MINCIENCIA'],
      dtype='object')


In [6]:
# Paso 1: Renombrar las columnas de institución
bdh_historica_df.rename(columns={'INSTITUCION_PRINCIPAL': 'Institución'}, inplace=True)
postulaciones_df.rename(columns={'TIPO_INSTITUCION': 'Institución'}, inplace=True)

# Paso 2: Realizar el merge con sufijos
bdh_postulaciones_merged = pd.merge(
    bdh_historica_df,
    postulaciones_df,
    on='CODIGO_PROYECTO',
    how='outer',
    suffixes=('_bdh', '_post')
)

# Paso 3: Eliminar columnas duplicadas
bdh_postulaciones_merged = bdh_postulaciones_merged.loc[:,~bdh_postulaciones_merged.columns.duplicated()]

# Paso 4: Crear la columna 'Institución' combinada
bdh_postulaciones_merged['Institución'] = bdh_postulaciones_merged['Institución_bdh'].combine_first(bdh_postulaciones_merged['Institución_post'])

# Paso 5: Aplicar la normalización y el mapeo
bdh_postulaciones_merged['Institución'] = bdh_postulaciones_merged['Institución'].apply(normalize_name)
bdh_postulaciones_merged['Institución'] = bdh_postulaciones_merged['Institución'].replace(institution_mapping)

# Paso 6: Crear la columna 'Año' en 'bdh_postulaciones_merged'
bdh_postulaciones_merged['AGNO_FALLO_bdh'] = pd.to_numeric(bdh_postulaciones_merged['AGNO_FALLO_bdh'], errors='coerce').astype('Int64')
bdh_postulaciones_merged['AGNO_FALLO_post'] = pd.to_numeric(bdh_postulaciones_merged['AGNO_FALLO_post'], errors='coerce').astype('Int64')
bdh_postulaciones_merged['Año'] = bdh_postulaciones_merged['AGNO_FALLO_bdh'].combine_first(bdh_postulaciones_merged['AGNO_FALLO_post'])

# Paso 7: Aplicar la normalización y el mapeo en 'consolidated_df'
consolidated_df['Institución'] = consolidated_df['Institución'].apply(normalize_name)
consolidated_df['Institución'] = consolidated_df['Institución'].replace(institution_mapping)

# Asegurar que 'Año' en 'consolidated_df' es numérico
consolidated_df['Año'] = pd.to_numeric(consolidated_df['Año'], errors='coerce').astype('Int64')

# Paso 8: Realizar el merge final
final_merged_df = pd.merge(
    consolidated_df,
    bdh_postulaciones_merged,
    on=['Institución', 'Año'],
    how='left',
    suffixes=('', '_bdh_post')
)

# Verifica las dimensiones del DataFrame final
print(final_merged_df.shape)




(5411656, 41)


In [7]:
print("Dimensiones del DataFrame final:", final_merged_df.shape)
print(final_merged_df.head())
print(final_merged_df.info())

Dimensiones del DataFrame final: (5411656, 41)
  Institución   Año                Dimensión      Indicador  \
0        unap  2022  1_ContextoInstitucional  Clasificación   
1        unap  2022  1_ContextoInstitucional  Clasificación   
2        unap  2022  1_ContextoInstitucional  Clasificación   
3        unap  2022  1_ContextoInstitucional  Clasificación   
4        unap  2022  1_ContextoInstitucional  Clasificación   

                               Valor    N_bdh CODIGO_PROYECTO  \
0  (a) Universidades Estatales CRUCH  38512.0        11221158   
1  (a) Universidades Estatales CRUCH  40050.0         1220290   
2  (a) Universidades Estatales CRUCH  38955.0         1221217   
3  (a) Universidades Estatales CRUCH  39322.0         1221784   
4  (a) Universidades Estatales CRUCH  38615.0         3220083   

             SUBDIRECCION_bdh PROGRAMA_bdh INSTRUMENTO_bdh  ...  \
0  PROYECTOS DE INVESTIGACION     FONDECYT      INICIACION  ...   
1  PROYECTOS DE INVESTIGACION     FONDECYT       

EDA

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [9]:
# Verificar el tamaño del DataFrame
print(f"El tamaño del DataFrame es: {final_merged_df.shape}")

# Primeras filas del DataFrame
print("Primeras filas del DataFrame:")
print(final_merged_df.head())

# Resumen estadístico
print("Resumen estadístico:")
print(final_merged_df.describe())

# Información de los tipos de datos y valores nulos
print("Información del DataFrame:")
print(final_merged_df.info())

# Ver valores nulos por columna
print("Valores nulos por columna:")
print(final_merged_df.isnull().sum())


El tamaño del DataFrame es: (5411656, 41)
Primeras filas del DataFrame:
  Institución   Año                Dimensión      Indicador  \
0        unap  2022  1_ContextoInstitucional  Clasificación   
1        unap  2022  1_ContextoInstitucional  Clasificación   
2        unap  2022  1_ContextoInstitucional  Clasificación   
3        unap  2022  1_ContextoInstitucional  Clasificación   
4        unap  2022  1_ContextoInstitucional  Clasificación   

                               Valor    N_bdh CODIGO_PROYECTO  \
0  (a) Universidades Estatales CRUCH  38512.0        11221158   
1  (a) Universidades Estatales CRUCH  40050.0         1220290   
2  (a) Universidades Estatales CRUCH  38955.0         1221217   
3  (a) Universidades Estatales CRUCH  39322.0         1221784   
4  (a) Universidades Estatales CRUCH  38615.0         3220083   

             SUBDIRECCION_bdh PROGRAMA_bdh INSTRUMENTO_bdh  ...  \
0  PROYECTOS DE INVESTIGACION     FONDECYT      INICIACION  ...   
1  PROYECTOS DE INVESTIG

EDA

In [10]:
# Tamaño del DataFrame
print(f"El DataFrame tiene {final_merged_df.shape[0]} filas y {final_merged_df.shape[1]} columnas.")

# Primeras filas del DataFrame
final_merged_df.head()


El DataFrame tiene 5411656 filas y 41 columnas.


,Institución,Año,Dimensión,Indicador,Valor,N_bdh,CODIGO_PROYECTO,SUBDIRECCION_bdh,PROGRAMA_bdh,INSTRUMENTO_bdh,...,INSTRUMENTO_post,NOMBRE_CONCURSO_post,ESTADO_RESOLUCION_CONCURSO,DISCIPLINA_OECD,REGION_EJECUCION_post,TIPO_BENEFICIARIO_post,Institución_post,SEXO_post,AGNO_FALLO_post,MACROZONA_MINCIENCIA_post
0,unap,2022,1_ContextoInstitucional,Clasificación,(a) Universidades Estatales CRUCH,38512.0,11221158,PROYECTOS DE INVESTIGACION,FONDECYT,INICIACION,...,INICIACION,INICIACION 2022,ADJUDICADO,CIENCIAS SOCIALES,REGIÓN DE TARAPACÁ,PERSONA NATURAL,PERSONA NATURAL,HOMBRE,2022,NORTE
1,unap,2022,1_ContextoInstitucional,Clasificación,(a) Universidades Estatales CRUCH,40050.0,1220290,PROYECTOS DE INVESTIGACION,FONDECYT,REGULAR,...,REGULAR,REGULAR 2022,ADJUDICADO,CIENCIAS SOCIALES,REGIÓN DE TARAPACÁ,PERSONA NATURAL,PERSONA NATURAL,HOMBRE,2022,NORTE
2,unap,2022,1_ContextoInstitucional,Clasificación,(a) Universidades Estatales CRUCH,38955.0,1221217,PROYECTOS DE INVESTIGACION,FONDECYT,REGULAR,...,REGULAR,REGULAR 2022,ADJUDICADO,CIENCIAS NATURALES,REGIÓN DE TARAPACÁ,PERSONA NATURAL,PERSONA NATURAL,HOMBRE,2022,NORTE
3,unap,2022,1_ContextoInstitucional,Clasificación,(a) Universidades Estatales CRUCH,39322.0,1221784,PROYECTOS DE INVESTIGACION,FONDECYT,REGULAR,...,REGULAR,REGULAR 2022,ADJUDICADO,CIENCIAS SOCIALES,REGIÓN DE TARAPACÁ,PERSONA NATURAL,PERSONA NATURAL,HOMBRE,2022,NORTE
4,unap,2022,1_ContextoInstitucional,Clasificación,(a) Universidades Estatales CRUCH,38615.0,3220083,PROYECTOS DE INVESTIGACION,FONDECYT,POSTDOCTORADO,...,POSTDOCTORADO,POSTDOCTORADO 2022,ADJUDICADO,CIENCIAS NATURALES,REGIÓN DE TARAPACÁ,PERSONA NATURAL,PERSONA NATURAL,HOMBRE,2022,NORTE


In [11]:
final_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5411656 entries, 0 to 5411655
Data columns (total 41 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Institución                 object 
 1   Año                         Int64  
 2   Dimensión                   object 
 3   Indicador                   object 
 4   Valor                       object 
 5   N_bdh                       float64
 6   CODIGO_PROYECTO             object 
 7   SUBDIRECCION_bdh            object 
 8   PROGRAMA_bdh                object 
 9   INSTRUMENTO_bdh             object 
 10  NOMBRE_CONCURSO_bdh         object 
 11  AGNO_CONCURSO               float64
 12  AGNO_FALLO_bdh              Int64  
 13  NOMBRE_PROYECTO             object 
 14  AREA_OCDE                   object 
 15  DISCIPLINA_DETALLE          object 
 16  GRUPO_DE_EVALUACION         object 
 17  DURACION_MESES              object 
 18  TIPO_BENEFICIARIO_bdh       object 
 19  NOMBRE_RESPONSABLE   

In [12]:
final_merged_df.describe()

,Año,N_bdh,AGNO_CONCURSO,AGNO_FALLO_bdh,NOTA_MONTO,N_post,AGNO_FALLO_post
count,5411656.0,5.067218e+06,5.066718e+06,5067218.0,0.0,2.536586e+06,2536586.0
mean,2015.299699,2.756489e+04,2.015112e+03,2015.377709,NaN,4.228613e+04,2018.829763
std,4.180792,7.260246e+03,4.208470e+00,4.155608,NaN,2.785329e+04,1.936831
min,2007.0,1.372600e+04,2.003000e+03,2007.0,NaN,1.000000e+00,2016.0
25%,2012.0,2.174800e+04,2.012000e+03,2012.0,NaN,1.520200e+04,2017.0
50%,2016.0,2.788300e+04,2.015000e+03,2016.0,NaN,4.476400e+04,2019.0
75%,2019.0,3.361600e+04,2.019000e+03,2019.0,NaN,6.161300e+04,2020.0
max,2022.0,4.037500e+04,2.022000e+03,2022.0,NaN,9.453400e+04,2022.0


In [13]:
final_merged_df.describe(include=['object'])


,Institución,Dimensión,Indicador,Valor,CODIGO_PROYECTO,SUBDIRECCION_bdh,PROGRAMA_bdh,INSTRUMENTO_bdh,NOMBRE_CONCURSO_bdh,NOMBRE_PROYECTO,...,PROGRAMA_post,INSTRUMENTO_post,NOMBRE_CONCURSO_post,ESTADO_RESOLUCION_CONCURSO,DISCIPLINA_OECD,REGION_EJECUCION_post,TIPO_BENEFICIARIO_post,Institución_post,SEXO_post,MACROZONA_MINCIENCIA_post
count,5411656,5411656,5411656,5168002,5067218,5067218,5067218,5067218,5067218,5067218,...,2536586,2536586,2536586,2536586,2536586,2536586,2536586,2536586,2536586,2536586
unique,224,10,116,13354,21938,6,17,200,742,21403,...,18,109,272,3,8,19,3,3,2,9
top,uchile,2_Estudiantes,Clasificación,0,SIN INFORMACION,PROYECTOS DE INVESTIGACION,FONDECYT,REGULAR,CONCURSO REGULAR FONDECYT 2014,SIN INFORMACION,...,FONDECYT,REGULAR,REGULAR,ADJUDICADO,CIENCIAS NATURALES,REGIÓN METROPOLITANA DE SANTIAGO,PERSONA NATURAL,PERSONA NATURAL,HOMBRE,RM
freq,1093066,928416,416974,316142,12344,3363718,3363718,1696138,132250,6036,...,1642540,815232,234462,2458918,931794,1436912,1779326,1779326,1719516,1436912


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
sns.heatmap(final_merged_df.isnull(), cbar=False, cmap='viridis')
plt.title('Mapa de calor de valores nulos')
plt.show()


In [ ]:
missing_percentage = final_merged_df.isnull().mean() * 100
print(missing_percentage.sort_values(ascending=False))
